## Import Modules

In [1]:
import requests
import pandas as pd
import time
import json
import pickle
import requests
from bs4 import BeautifulSoup  
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

## Scrape Data 

#### Cordcutters Subreddit

In [2]:
url_cord = 'https://www.reddit.com/r/cordcutters.json'

In [3]:
posts_cord = []
after = None 

for i in range(40):
    if after == None: 
        current_url = url_cord
    else:
        current_url = url_cord + '?after=' + after
    print(current_url)
    res = requests.get(current_url, headers={'User-agent': 'Cordcutters'})
    if res.status_code != 200:
        print('Status error', res.status_code)
        break
    current_dict = res.json()
    current_post = [p['data'] for p in current_dict['data']['children']]
    posts_cord.extend(current_post)
    after = current_dict['data']['after'] 
    if after == None:
        break

    time.sleep(1)

https://www.reddit.com/r/cordcutters.json
https://www.reddit.com/r/cordcutters.json?after=t3_9e64jf
https://www.reddit.com/r/cordcutters.json?after=t3_9e1o6h
https://www.reddit.com/r/cordcutters.json?after=t3_9domi2
https://www.reddit.com/r/cordcutters.json?after=t3_9dc5s2
https://www.reddit.com/r/cordcutters.json?after=t3_9cwflo
https://www.reddit.com/r/cordcutters.json?after=t3_9cmjvp
https://www.reddit.com/r/cordcutters.json?after=t3_9by5yy
https://www.reddit.com/r/cordcutters.json?after=t3_9btuk7
https://www.reddit.com/r/cordcutters.json?after=t3_9bpgho
https://www.reddit.com/r/cordcutters.json?after=t3_9bgs3z
https://www.reddit.com/r/cordcutters.json?after=t3_9b45fd
https://www.reddit.com/r/cordcutters.json?after=t3_9apfxf
https://www.reddit.com/r/cordcutters.json?after=t3_9ai7nx
https://www.reddit.com/r/cordcutters.json?after=t3_99xocs
https://www.reddit.com/r/cordcutters.json?after=t3_99rgvi
https://www.reddit.com/r/cordcutters.json?after=t3_99gvm4
https://www.reddit.com/r/cordc

#### Import Cordcutters Data

In [4]:
with open('data/cordcutters_list', 'w') as f:
    json.dump(posts_cord, f) 

In [5]:
with open('data/cordcutters_list', 'r') as f:
    post_cords_from_json = json.loads(f.read())

In [6]:
print('cc dict count:', len(post_cords_from_json))

cc dict count: 933


#### Piracy Subreddit

In [7]:
piracy_url = 'https://www.reddit.com/r/Piracy.json'

In [8]:
posts_pir = []
after = None 

for i in range(40):
    if after == None: 
        current_url = piracy_url
    else:
        current_url = piracy_url + '?after=' + after
    print(current_url)
    res = requests.get(current_url, headers={'User-agent': 'Piracy'})
    if res.status_code != 200:
        print('Status error', res.status_code)
        break
    current_dict = res.json()
    current_post = [p['data'] for p in current_dict['data']['children']]
    posts_pir.extend(current_post)
    after = current_dict['data']['after'] 
    if after == None:
        break

    time.sleep(1)

https://www.reddit.com/r/Piracy.json
https://www.reddit.com/r/Piracy.json?after=t3_9emy1v
https://www.reddit.com/r/Piracy.json?after=t3_9egfny
https://www.reddit.com/r/Piracy.json?after=t3_9ee0sh
https://www.reddit.com/r/Piracy.json?after=t3_9ecfru
https://www.reddit.com/r/Piracy.json?after=t3_9dybl1
https://www.reddit.com/r/Piracy.json?after=t3_9dy48q
https://www.reddit.com/r/Piracy.json?after=t3_9dqsz8
https://www.reddit.com/r/Piracy.json?after=t3_9dk2l5
https://www.reddit.com/r/Piracy.json?after=t3_9ddnzk
https://www.reddit.com/r/Piracy.json?after=t3_9d2tkx
https://www.reddit.com/r/Piracy.json?after=t3_9cy7aw
https://www.reddit.com/r/Piracy.json?after=t3_9calsh
https://www.reddit.com/r/Piracy.json?after=t3_9ckdv9
https://www.reddit.com/r/Piracy.json?after=t3_9cezns
https://www.reddit.com/r/Piracy.json?after=t3_9c3cdb
https://www.reddit.com/r/Piracy.json?after=t3_9c559s
https://www.reddit.com/r/Piracy.json?after=t3_9c47p2
https://www.reddit.com/r/Piracy.json?after=t3_9c0y7v
https://w

#### Import Piracy Data

In [9]:
with open('data/piracy_list', 'w') as f:
    json.dump(posts_pir, f) 

In [10]:
with open('data/piracy_list', 'r') as f:
    post_pir_from_json = json.loads(f.read())

In [11]:
print('pir dict count:', len(post_pir_from_json))

pir dict count: 636


## Convert JSON Data into DataFrame

In [12]:
def posts_to_df(posts, features = ['subreddit', 'author', 'title', 'selftext', 'created_utc', 'num_comments']):
    feat_dict = [{feat : post[feat] for feat in features}  for post in posts]
    return pd.DataFrame(feat_dict)

In [13]:
feature_list = ['subreddit', 'author', 'title', 'selftext', 'created_utc', 'num_comments','score','over_18',
                'score']
df_cordcutters = posts_to_df(post_cords_from_json, features=feature_list)

df_cordcutters.head()

,author,created_utc,num_comments,over_18,score,selftext,subreddit,title
0,frostcall,1.533159e+09,129,False,148,"For greater details on this topic, please refe...",cordcutters,NFL viewing guide for Cord Cutter's
1,VaritasAequitas,1.536582e+09,21,False,46,I know this question gets thrown around consta...,cordcutters,I want to get my family to stop renting the Co...
2,blaspheminCapn,1.536587e+09,0,False,7,,cordcutters,Top 7 Free TV Shows Streaming Sites To Watch T...
3,alexlfire,1.536516e+09,81,False,145,I'm going to get my tin foil hat out. I live w...,cordcutters,Antenna cuts out only during NFL games
4,fatmalakas,1.536522e+09,24,False,65,Frustrated cord-cutter here...need some help w...,cordcutters,Just set up attic Clearstream 2V w/ pre-amp bu...


In [14]:
df_piracy = posts_to_df(post_pir_from_json, features=feature_list)

df_piracy.head()

,author,created_utc,num_comments,over_18,score,selftext,subreddit,title
0,AutoModerator,1.536502e+09,44,False,9,This thread is for the r/Piracy community to d...,Piracy,Weekly General Discussion Thread (September 09...
1,WinterRise,1.536581e+09,25,False,310,,Piracy,Laughs in Piracy
2,Drakantas,1.536522e+09,107,False,1412,,Piracy,Seeding explained
3,MON5TERMATT,1.536556e+09,10,False,37,Due to the guides put out by users I have mana...,Piracy,Thank you r/piracy.
4,DARKFIRE1819,1.536484e+09,408,False,997,,Piracy,I hope my pc is ok


#### Concatenate the DataFrames

In [15]:
df = pd.concat([df_cordcutters, df_piracy], axis=0, ignore_index=True)

#### Create a Boolean Column

In [16]:
df['label'] = df['subreddit'].map(lambda x: 1 if x == 'cordcutters' else 0)

In [17]:
df.head()

,author,created_utc,num_comments,over_18,score,selftext,subreddit,title,label
0,frostcall,1.533159e+09,129,False,148,"For greater details on this topic, please refe...",cordcutters,NFL viewing guide for Cord Cutter's,1
1,VaritasAequitas,1.536582e+09,21,False,46,I know this question gets thrown around consta...,cordcutters,I want to get my family to stop renting the Co...,1
2,blaspheminCapn,1.536587e+09,0,False,7,,cordcutters,Top 7 Free TV Shows Streaming Sites To Watch T...,1
3,alexlfire,1.536516e+09,81,False,145,I'm going to get my tin foil hat out. I live w...,cordcutters,Antenna cuts out only during NFL games,1
4,fatmalakas,1.536522e+09,24,False,65,Frustrated cord-cutter here...need some help w...,cordcutters,Just set up attic Clearstream 2V w/ pre-amp bu...,1


In [18]:
df.shape

(1569, 9)

In [19]:
df.label.value_counts(normalize=True)

1    0.594646
0    0.405354
Name: label, dtype: float64

In [20]:
df.isnull().sum()

author          0
created_utc     0
num_comments    0
over_18         0
score           0
selftext        0
subreddit       0
title           0
label           0
dtype: int64

In [21]:
df.duplicated().sum()

0

## Processing Text Data

In [22]:
df['selftext'] = df['selftext'].replace(r'[^a-zA-Z0-9\s\/]', '', regex=True)

df['selftext'] = df['selftext'].map(lambda x: x.lower())

df['selftext'] = df['selftext'].replace(r'[\r\n]+', ' ', regex=True)

df['selftext'] = df['selftext'].str.replace('\d+', '')

df['title'] = df['title'].replace(r'[^a-zA-Z0-9\s\/]', '', regex=True)

df['title'] = df['title'].map(lambda x: x.lower())

df['title'] = df['title'].replace(r'[\r\n]+', ' ', regex=True)

df['title'] = df['title'].str.replace('\d+', '')

## Create  and Export Full Stopword List

In [23]:
url = 'https://raw.githubusercontent.com/stanfordnlp/CoreNLP/master/data/edu/stanford/nlp/patterns/surface/stopwords.txt'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')

In [24]:
standford_nlp = soup.text.split('\n')

In [25]:
stop_word_list = standford_nlp + list(ENGLISH_STOP_WORDS)

## Export Clean DataFrame for EDA

In [26]:
with open('./Data/clean_df.pkl', 'wb+') as f:
    pickle.dump(df, f)

## Export Stopwords List

In [27]:
with open('./Data/stop_word_list.pkl', 'wb+') as f:
    pickle.dump(stop_word_list, f)